In [1]:
# We will do the imports required for this notebook here
%load_ext autoreload
%autoreload 2
# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# swift is a lightweight browser-based simulator which comes eith the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# import urchin to read urdf files
from urchin import URDF

# this package provides several solvers for solving quadratic programmes
import qpsolvers as qp 

# modern robotics code library accompanying 
# Modern Robotics: Mechanics, Planning, and Control (Kevin Lynch and Frank Park)
import modern_robotics as mr

# the os package provides utilities for interacting with the operating system
import os

# import config 
from config.config import * 
from models.panda_model import *

import pytransform3d as pt3d

In [10]:
R = sm.SO3.RPY(np.pi/3, 0, np.pi/6)
start = SE3.Rt(R, np.array([0.1, 0, 0]))

start.inv()

   0.866     0.5       0        -0.0866    
  -0.25      0.433     0.866     0.025     
   0.433    -0.75      0.5      -0.0433    
   0         0         0         1         


In [47]:
radius = 0.037
height = 0.234

cylinder = sg.Cylinder(radius=radius, length=height, 
                       pose=SE3(0.1, 0, height/2) * SE3.Rx(np.pi/6), color='purple')
cylinder.set_alpha(0.5)
env = Swift()
wTc = sg.Axes(length=0.1)
wTc.T = cylinder.T
env.launch(realtime=True, browser="notebook")
env.add(cylinder)
env.add(wTc)
env.add(box)

2

In [50]:
sm.SO3(SE3(cylinder.T).R) * np.array([0, 0, 1])

array([[ 0.       ],
       [-0.5      ],
       [ 0.8660254]])